In [1]:
import numpy as np
import pandas as pd
import PIL
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,LocallyConnected2D,BatchNormalization,Conv2DTranspose
from tensorflow.keras.preprocessing import image

In [2]:
attr_data_path = "list_attr_celeba.csv"
images_data_path = "C:/Users/foreh/Documents/celeba/greyscale"
images_data_size = 10000 

tf.keras.backend.clear_session()
width = 178
height = 218
x_ = 100                            # new value of x
image_size_ratio = width / height    # dimensions ratio
y_ = int(image_size_ratio * x_)  
model = Sequential()
model.add(tf.keras.Input(shape=(height, width,1)))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding = 'valid', kernel_regularizer='l2'))
model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), padding='valid', kernel_regularizer='l2'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(3, 3), padding='same', kernel_regularizer='l2'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))

model.add(Conv2D(filters=32,kernel_size=(3,3),padding = 'valid', kernel_regularizer='l2'))
model.add(Conv2D(filters=8,kernel_size=(2,2),padding = 'valid', kernel_regularizer='l2'))
model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(filters=4, kernel_size=(3, 3), padding='same', kernel_regularizer='l2'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))

model.add(Conv2D(filters=16,kernel_size=(3,3),padding = 'same', kernel_regularizer='l2'))
model.add(Conv2D(filters=8,kernel_size=(3,3),padding = 'same', kernel_regularizer='l2'))
model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(filters=4, kernel_size=(3, 3), padding='same', kernel_regularizer='l2'))
model.add(MaxPooling2D(pool_size=4))

model.add(Conv2D(filters=16, kernel_size=(3,3)))
model.add(Conv2D(filters=16, kernel_size=(2,2)))
model.add(tf.keras.layers.AveragePooling2D(pool_size=2))
model.add(Conv2D(filters=8, kernel_size=(3,3)))
model.add(tf.keras.layers.BatchNormalization())

model.add(Flatten())
model.add(Dense(64))
model.add(Dense(120))
model.add(Dense(48))
model.add(Dense(16))
model.add(Dense(1, activation='sigmoid'))
original_image_size = (width,height)
df_org = pd.read_csv(attr_data_path)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 218, 178, 1)      4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 216, 176, 32)      320       
                                                                 
 conv2d_1 (Conv2D)           (None, 214, 174, 8)       2312      
                                                                 
 batch_normalization_1 (Batc  (None, 214, 174, 8)      32        
 hNormalization)                                                 
                                                                 
 conv2d_transpose (Conv2DTra  (None, 214, 174, 128)    9344      
 nspose)                                                         
                                                        

In [3]:
for i in range(1, 100):
    if (i > 1):
        model = tf.keras.models.load_model('cnn83_checkpoints/iter'+str(i-1))
        print("model loaded: iter"+str(i-1))
        print("training model "+str(i))
    df = pd.DataFrame()
    if((i+1) % 10 == 0):
        df = df_org[(1+images_data_size*(i%20)):202600]
    else:
        df = df_org[(1+images_data_size*(i%20)):images_data_size*((i+1)%20)]
    df = df.replace(-1,0)
    #print(df.head(5))
    images_data = list()
    for idx in range(df.shape[0]):
        path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
        image = PIL.Image.open(path)
        image_array = np.asarray(image)
        images_data.append(image_array)
    images_data = np.array(images_data)
    def image_from_index(index, size=(original_image_size), path=images_data_path):
        path = "{}/{}".format(str(path),str(df.iloc[index].image_id))
        # to read the image
        image = PIL.Image.open(path).resize(size)
        image_array = np.asarray(image)

        return image_array
    images = images_data # list of array of images
    labels = df[["image_id", "Bangs"]]#, "Straight_Hair", "Wavy_Hair", "Wearing_Hat"]]  # dataframe of image features
    train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=(i%6))
    y_test=test_labels.drop(["image_id"],axis=1)
    y_train=train_labels.drop(["image_id"],axis=1)

    X_test=test_images
    X_train=train_images
    #X_train = tf.expand_dims(X_train, axis=-1)
    print("Samples Training Set:",len(X_train))
    X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=(i%5))
    print("Samples in Validation Data:",len(X_val))
    print("Samples in Test Data:",len(X_test))
    x_ = original_image_size[0]
    y_ = original_image_size[1]
    BATCH_SIZE = (8)
    print("Batch size: " + str(BATCH_SIZE))
    if True or i%3 == 1:
          model.compile(optimizer = "adam",loss="binary_crossentropy",metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 'binary_accuracy'])
    elif True or i%3 == 2 or i%3 == 0:
          print("SGD")
          opt = tf.keras.optimizers.SGD(learning_rate=0.001)
          model.compile(optimizer = opt,loss="binary_crossentropy",metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 'binary_accuracy'])
    else:
          print("ADAM")
          opt = tf.keras.optimizers.Adam(learning_rate=0.001)
          model.compile(optimizer = opt,loss="binary_crossentropy",metrics= ["binary_accuracy"])#tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 'binary_accuracy'])
    training_process = model.fit(X_train,y_train,epochs = 5,validation_data = (X_val,y_val),batch_size=BATCH_SIZE,shuffle = True)
    model.save('cnn83_checkpoints/iter'+str(i))
    print("Model " + str(i) +"evaluating:")
    model.evaluate(X_val,y_val)
    tf.keras.backend.clear_session()

Samples Training Set: 6999
Samples in Validation Data: 1500
Samples in Test Data: 1500
Batch size: 8
Epoch 1/5
875/875 [==============================] - 65s 70ms/step - loss: 0.7452 - precision: 0.7061 - recall: 0.5532 - binary_accuracy: 0.8946 - val_loss: 0.4473 - val_precision: 0.7531 - val_recall: 0.4880 - val_binary_accuracy: 0.8880
Epoch 2/5
875/875 [==============================] - 61s 70ms/step - loss: 0.3300 - precision: 0.7574 - recall: 0.5844 - binary_accuracy: 0.9061 - val_loss: 0.3201 - val_precision: 0.8737 - val_recall: 0.3320 - val_binary_accuracy: 0.8807
Epoch 3/5
875/875 [==============================] - 62s 70ms/step - loss: 0.2499 - precision: 0.7995 - recall: 0.6477 - binary_accuracy: 0.9198 - val_loss: 0.2674 - val_precision: 0.7488 - val_recall: 0.6320 - val_binary_accuracy: 0.9033
Epoch 4/5
875/875 [==============================] - 62s 71ms/step - loss: 0.2683 - precision: 0.7751 - recall: 0.6101 - binary_accuracy: 0.9117 - val_loss: 0.3182 - val_precision: 0

KeyboardInterrupt: 

In [ ]:
attr_data_path = "list_attr_celeba.csv"
images_data_path = "C:/Users/foreh/Documents/celeba/greyscale"

tf.keras.backend.clear_session()
width = 178
height = 218
df_org = pd.read_csv(attr_data_path)

df = df_org[190000:202600]
df = df.replace(-1,0)
#print(df.head(5))
images_data = list()
for idx in range(df.shape[0]):
    path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
    image = PIL.Image.open(path)
    image_array = np.asarray(image)
    images_data.append(image_array)
images_data = np.array(images_data)
def image_from_index(index, size=(width,height), path=images_data_path):
    path = "{}/{}".format(str(path),str(df.iloc[index].image_id))
    # to read the image
    image = PIL.Image.open(path).resize(size)
    image_array = np.asarray(image)

    return image_array
images = images_data # list of array of images
labels = df[["image_id", "Bangs", "Straight_Hair", "Wavy_Hair", "Wearing_Hat"]]  # dataframe of image features
labels=labels.drop(["image_id"],axis=1)



In [ ]:
for i in range(1, 48):
    tf.keras.backend.clear_session()
    print("###### EVAL MODEL "+str(i)+"######")
    model1 = tf.keras.models.load_model('cnn82_checkpoints/iter'+str(i))
    model1.evaluate(images,labels)